Compute some values from Spinorama following Olive&al

In [4]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker
from src.spinorama.graph import graph_params_default

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [5]:
def octave(N):
    """compute 1/N octave band"""
    p = pow(2,1/N)
    p_band= pow(2,1/(2*N))
    iter = int((N*10+1)/2)
    center = [1000 / p**i for i in range(iter,0,-1)]+[1000*p**i for i in range(0,iter,1)]
    return [(c/p_band,c*p_band) for c in center]

octave20 = octave(20)




In [16]:
def aad(dfu):
    # mean betwenn 200hz and 400hz
    y_ref = np.mean(dfu.loc[(dfu.Freq>=200) & (dfu.Freq<=400)].dB)
    #print(y_ref)
    sum = 0
    n = 0
    # 1/20 octave
    for (omin, omax) in octave(20):
        # 100hz to 16k hz
        if omin < 100:
            continue
        if omax > 16000:
            break
        sum += abs(y_ref-np.mean(dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)].dB))
        n += 1
    return sum/n

def nbd(dfu):
    sum = 0
    n = 0
    # 1/2 octave
    for (omin, omax) in octave(2):
        # 100hz to 12k hz
        if omin < 100:
            continue
        if omax > 12000:
            break
        y = dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)].dB
        y_avg = np.mean(y)
        # don't sample, take all points in this octave
        sum += np.mean(np.abs(y_avg-y))
        n += 1
    return sum/n

def lfx(lw, sp):
    y_ref = np.mean(lw.loc[(lw.Freq>=300) & (lw.Freq<=10000)].dB)-6
    # find first freq such that y[freq]<y_ref-6dB
    y = math.log10(sp.loc[(sp.Freq<300)&(sp.dB<y_ref)].Freq.max())
    return y

def lfq(lw, sp, lfx_log):
    lfx = pow(10,lfx_log)
    sum = 0
    n = 0
    for (omin, omax) in octave(20):
        # 100hz to 12k hz
        if omin < lfx:
            continue
        if omax > 300:
            break
        y_lw = np.mean(lw.loc[(lw.Freq>=omin) & (lw.Freq<omax)].dB)
        y_sp = np.mean(sp.loc[(sp.Freq>=omin) & (sp.Freq<omax)].dB)
        sum += abs(y_lw-y_sp)
        n += 1
    return sum/n
    
df_onaxis = df['CEA2034'].loc[lambda df: df.Measurements == 'On Axis']
df_lw = df['CEA2034'].loc[lambda df: df.Measurements == 'Listening Window']
df_sp = df['CEA2034'].loc[lambda df: df.Measurements == 'Sound Power']

print('aad={0}'.format(aad(df_onaxis)))
print('nbd={0}'.format(nbd(df_onaxis)))
lfx_value = lfx(df_lw, df_sp)
print('lfx={0}'.format(lfx_value))
print('lfq={0}'.format(lfq(df_lw, df_sp, lfx_value)))

aad=0.6374662068965525
nbd=0.32108885482643135
lfx=1.722094009070862
lfq=0.3184897959183667
